In [1]:
import os
import pandas as pd
import numpy as np

import glob
import cv2
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input


if str(os.getcwdb()[-3:]).split("'")[1] != 'src':
    for _ in range(2):
        os.chdir(os.path.dirname(os.getcwdb()))


In [3]:
processed_images_path = r'data\processed\images'
df_images_data = pd.read_csv(r'data\processed\images_data_processed.csv')

df_images_data.head()


,Id,weight (carat),cut quality,color quality,clarity quality,depth (percentage),length (millimeters),width (millimeters),depth (millimeters)
0,1638147,0.55,4.0,5.0,1.0,62.553191,5.05,4.35,2.94
1,1612606,0.51,4.0,2.0,3.0,64.900662,4.71,4.35,2.94
2,1638140,0.50,4.0,2.0,3.0,62.813522,4.91,4.26,2.88
3,1536093,0.53,4.0,6.0,2.0,65.720524,4.70,4.46,3.01
4,1643527,0.52,4.0,1.0,6.0,65.141612,4.76,4.42,2.99


# Tamaño de las imágenes

- Se modifican los píxeles de cada imagen, de 300 a 224, para que puedan encajar en el modelo

In [4]:
for image_path in glob.glob(processed_images_path+'/*.jpg'):
    with Image.open(image_path) as image:
        image = image.resize((224, 224))
        image.save(image_path)


In [5]:
# Se comprueba que el cambio ha surtido efecto
for image in glob.glob(processed_images_path+'/*.jpg'):
    image_matrix = cv2.imread(image)
    break

image_matrix.shape


(224, 224, 3)

# "Split"

- Se separa el "dataframe" en "train" y "test"

In [6]:
df_images_data['Id'] = df_images_data['Id'].apply(lambda x: x + '.jpg')

df_images_data.head()


,Id,weight (carat),cut quality,color quality,clarity quality,depth (percentage),length (millimeters),width (millimeters),depth (millimeters)
0,1638147.jpg,0.55,4.0,5.0,1.0,62.553191,5.05,4.35,2.94
1,1612606.jpg,0.51,4.0,2.0,3.0,64.900662,4.71,4.35,2.94
2,1638140.jpg,0.50,4.0,2.0,3.0,62.813522,4.91,4.26,2.88
3,1536093.jpg,0.53,4.0,6.0,2.0,65.720524,4.70,4.46,3.01
4,1643527.jpg,0.52,4.0,1.0,6.0,65.141612,4.76,4.42,2.99


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_images_data['Id'], df_images_data.drop(columns='Id'), train_size=0.8, random_state=42)

df_train = pd.concat((X_train, y_train), axis=1)
df_test = pd.concat((X_test, y_test), axis=1)

df_train.head()


,Id,weight (carat),cut quality,color quality,clarity quality,depth (percentage),length (millimeters),width (millimeters),depth (millimeters)
3778,1798065.jpg,0.50,4.0,0.0,3.0,61.825319,5.08,5.11,3.15
978,1786532.jpg,0.31,4.0,5.0,5.0,51.371571,4.89,3.13,2.06
251,1634076.jpg,0.80,4.0,1.0,2.0,64.947469,5.43,5.04,3.40
2154,1643658.jpg,0.59,4.0,3.0,5.0,47.773973,7.13,4.55,2.79
4099,1769140.jpg,0.50,4.0,5.0,1.0,59.323671,5.17,5.18,3.07


# "Data augmentation"

- Se crea una variable para generar imágenes en diferentes posiciones para que el modelo disponga del mismo diamante colocado de modos distintos


In [9]:
# Se establecen las variables para crear nuevos diamantes y para seleccionar el tamaño de imagen correcto
data_augmentation = ImageDataGenerator(rotation_range=20,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        preprocessing_function=preprocess_input,
                                        validation_split=0.1
                                        )


In [10]:
# Se crean tres "generators" con los datos aumentados (entrenamiento, validación y "test")
train_generator = data_augmentation.flow_from_dataframe(dataframe=df_train,
                                                        directory=processed_images_path,
                                                        target_size=(224, 224),
                                                        class_mode='raw',
                                                        shuffle=False,
                                                        x_col='Id',
                                                        y_col=list(df_images_data.columns[1:]),
                                                        seed=42,
                                                        subset='training'
                                                        )

validation_generator = data_augmentation.flow_from_dataframe(dataframe=df_train,
                                                                directory=processed_images_path,
                                                                target_size=(224, 224),
                                                                class_mode='raw',
                                                                shuffle=False,
                                                                x_col='Id',
                                                                y_col=list(df_images_data.columns[1:]),
                                                                seed=42,
                                                                subset='validation'
                                                                )

test_generator = data_augmentation.flow_from_dataframe(dataframe=df_test,
                                                        directory=processed_images_path,
                                                        target_size=(224, 224),
                                                        class_mode='raw',
                                                        shuffle=False,
                                                        x_col='Id',
                                                        y_col=list(df_images_data.columns[1:]),
                                                        seed=42,
                                                        )


Found 3254 validated image filenames.
Found 361 validated image filenames.
Found 904 validated image filenames.


# Modelaje: entreno solo con imágenes

- Se elige VGG16 por las siguientes razones:

1) Es popular y se ha utilizado con éxito en investigación
2) Es relativamente fácil utilizarlo para problemas de regresión
3) Tiene un buen rendimiento
4) Trabaja con RGB, y el color de los diamantes es importante
5) Utiliza un tamaño de 224x224, y se ha visto que a partir de 150 componentes se obtiene toda la información necesaria
6) Otros como ResNet, Inception y EfficientNet son más modernos y podrían dar mejores resultados, pero el coste computacional podría elevarse mucho, lo que con toda probabilidad no valdría la pena para un "dataset" tan pequeño
7) Se ve durante el "train" que, a pesar de ser pocas las fotos, dado que son muchas las variables a predecir y se trata de un problema de regressión, el modelo es extremadamente lento. Por tanto, es inviable probar con los que son mas complejos, pues tardarían incluso más

- El modelo sufre "overfitting"

In [12]:
# Se carga el modelo sin la capa superior
base_model = VGG16(include_top=False, input_shape=((224, 224, 3)))


In [105]:
# Se congelan las capas base para que no se entrenen al tunear parámetros, sino que queden igual
for layer in base_model.layers:
    layer.trainable = False


In [106]:
# Se crea una nueva capa superior
top_model = Flatten()(base_model.output)
top_model = Dense(1024, activation='relu')(top_model)
top_model = Dense(512, activation='relu')(top_model)
output_layer = Dense(8, activation='linear')(top_model)


In [107]:
# Se tunea el modelo con la capa nueva
model = Model(inputs=base_model.input, outputs=output_layer)


In [108]:
# Se compila y se le pone un optimizador
model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')


In [109]:
# Se entrena el modelo con "early stopping"
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_generator,
                    epochs=20,
                    batch_size=64,
                    validation_data=validation_generator,
                    callbacks=[early_stop]
                    )


Epoch 1/20
102/102 [==============================] - 2804s 27s/step - loss: 10.4851 - val_loss: 9.5512
Epoch 2/20
102/102 [==============================] - 2877s 28s/step - loss: 9.1146 - val_loss: 8.5894
Epoch 3/20
102/102 [==============================] - 2878s 28s/step - loss: 8.1391 - val_loss: 7.5508
Epoch 4/20
102/102 [==============================] - 2717s 27s/step - loss: 7.2157 - val_loss: 6.8317
Epoch 5/20
102/102 [==============================] - 2716s 27s/step - loss: 6.3320 - val_loss: 5.6712
Epoch 6/20
102/102 [==============================] - 2717s 27s/step - loss: 4.8076 - val_loss: 3.7442
Epoch 7/20
102/102 [==============================] - 2717s 27s/step - loss: 2.6211 - val_loss: 1.7659
Epoch 8/20
102/102 [==============================] - 2716s 27s/step - loss: 1.5800 - val_loss: 1.5317
Epoch 9/20
102/102 [==============================] - 2716s 27s/step - loss: 1.5183 - val_loss: 1.5358
Epoch 10/20
102/102 [==============================] - 2716s 27s/step - 

In [113]:
y_pred = model.predict(test_generator)


29/29 [==============================] - 737s 25s/step


In [128]:
# Se obtienen las métricas
metrics_dict = dict()
for index, col in enumerate(df_images_data.columns[1:]):
    rmse = mean_squared_error([row[index] for row in test_generator.labels], [row[index] for row in y_pred], squared=False)
    mse = mean_squared_error([row[index] for row in test_generator.labels], [row[index] for row in y_pred])
    mae = mean_absolute_error([row[index] for row in test_generator.labels], [row[index] for row in y_pred])
    r2 = r2_score([row[index] for row in test_generator.labels], [row[index] for row in y_pred])
    mape = mean_absolute_percentage_error([row[index] for row in test_generator.labels], [row[index] for row in y_pred])
    metrics_dict[col] = {'mse': mse,
                            'rmse': rmse,
                            'mae': mae,
                            'r2': r2,
                            'mape': mape
                         }

metrics_dict


{'weight (carat)': {'mse': 0.13328463615810393,
  'rmse': 0.3650816842271109,
  'mae': 0.19201657599052496,
  'r2': -0.03231979506041949,
  'mape': 0.2904260471125496},
 'cut quality': {'mse': 1.5673639774311583,
  'rmse': 1.2519440791948968,
  'mae': 0.673318200406775,
  'r2': -0.406961150657724,
  'mape': 99986541709004.31},
 'color quality': {'mse': 9.108733336964494,
  'rmse': 3.018067815169913,
  'mae': 2.2640291167571482,
  'r2': -0.04341466119183379,
  'mape': 1203378812593506.2},
 'clarity quality': {'mse': 2.28118527908921,
  'rmse': 1.5103593211846016,
  'mae': 1.1890781432126476,
  'r2': -0.003099937064111158,
  'mape': 60004893071016.19},
 'depth (percentage)': {'mse': 59.99285645714846,
  'rmse': 7.745505564980795,
  'mae': 5.830462979976336,
  'r2': -0.042812788555836256,
  'mape': 0.10563365622823222},
 'length (millimeters)': {'mse': 1.5820806109324623,
  'rmse': 1.2578078593062068,
  'mae': 0.829696589275799,
  'r2': -0.05577155905441877,
  'mape': 0.1395914518773208},

In [131]:
# Se visualiza la predicción del peso, que es la más importante, en un "dataframe"
df_weight = pd.DataFrame(data={'original weight': [row[0] for row in test_generator.labels], 'Prediction': [row[0] for row in y_pred]})

df_weight


,original weight,Prediction
0,0.50,0.535823
1,1.61,0.535823
2,0.50,0.535823
3,0.75,0.535823
4,0.50,0.535823
...,...,...
899,0.50,0.535823
900,0.50,0.535823
901,0.31,0.535823
902,0.50,0.535823


# Modelaje: entreno con imágenes y tamaño

- Como las métricas anteriores no son muy buenas, se entrena el modelo pasándole, esta vez, las variables de tamaño, pues son las más sencillas de medir uno mismo en casa y, de ellas, puede extraerse también el peso